# Importations

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import cross_val_score
import math
from tensorflow.keras import Sequential

In [5]:
data = pd.read_csv("../raw_data/data_final_indé_big.csv")

In [6]:
data

,steam_appid,name,top_5_tags,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,...,english,price,has_a_website,nb_review,owner_estimated,rating,popularity,score_descriptif,average_playtime,median_playtime
0,1002,Rag Doll Kung Fu,fighting multiplayer 1980s realistic real_time...,0.016212,0.016212,0.016212,0.016212,0.401611,0.016212,0.016212,...,1,5.99,1,57,22451.0,29.800000,45.100000,9,0,0
1,2710,Act of War: Direct Action,action strategy rts 1980s relaxing,0.016444,0.016444,0.016444,0.016444,0.016444,0.016444,0.016444,...,1,1.99,0,213,29159.0,1522.087755,1924.966327,4,0,0
2,2720,ThreadSpace: Hyperbol,action strategy space 1980s relaxing,0.016197,0.016197,0.016197,0.016197,0.016197,0.016197,0.016197,...,1,3.99,1,40,21720.0,225.287500,276.278125,9,0,0
3,2780,ARMA: Combat Operations,action simulation shooter fps tactical,0.016200,0.016200,0.016200,0.016200,0.431614,0.016200,0.016200,...,1,5.99,1,222,29546.0,10759.884348,15763.549130,8,0,0
4,2850,X: Tension,simulation strategy space sci_fi space_sim,0.016250,0.016250,0.016250,0.016250,0.016250,0.016250,0.016250,...,1,3.49,1,75,23225.0,629.000000,1339.437500,6,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21542,1065230,Room of Pandora,adventure casual puzzle 1980s relaxing,0.015694,0.701810,0.015694,0.015694,0.015694,0.015694,0.015694,...,1,2.09,0,3,624.0,3.000000,3.000000,1,0,0
21543,1065570,Cyber Gun,action adventure fps 3d_platformer fast_paced,0.016478,0.016478,0.519926,0.016478,0.183470,0.016478,0.016478,...,1,1.69,0,9,1872.0,13.366667,16.816667,3,0,0
21544,1065650,Super Star Blast,action casual 1980s relaxing realistic,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,0.015483,...,1,3.99,1,1,208.0,4.600000,10.450000,5,0,0
21545,1066700,New Yankee 7: Deer Hunters,casual adventure 1980s relaxing realistic,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,0.015466,...,1,5.19,1,2,416.0,23.463964,29.301802,2,0,0


In [7]:
data.average_playtime.value_counts()

0      19736
1         89
2         38
3         34
9         28
       ...  
275        1
445        1
119        1
912        1
154        1
Name: average_playtime, Length: 571, dtype: int64

In [8]:
obj = data.isnull().sum()
for key,value in obj.iteritems():
    print(f'"{key}"')

"steam_appid"
"name"
"top_5_tags"
"topic_0"
"topic_1"
"topic_2"
"topic_3"
"topic_4"
"topic_5"
"topic_6"
"topic_7"
"topic_8"
"topic_9"
"topic_10"
"topic_11"
"topic_12"
"topic_13"
"topic_14"
"topic_15"
"topic_16"
"topic_17"
"topic_18"
"topic_19"
"english"
"price"
"has_a_website"
"nb_review"
"owner_estimated"
"rating"
"popularity"
"score_descriptif"
"average_playtime"
"median_playtime"


In [11]:
X = data.drop(columns=['owner_estimated'])
y = data['owner_estimated']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [13]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(iterations=2,
                          learning_rate=1,
                          depth=2)
# Fit model
model.fit(X_train, y_train)
# Get predictions
preds = model.predict(X_test)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=1]="Entropy Rising": Cannot convert 'b'Entropy Rising'' to float

# Baseline model

In [ ]:
resultat = 0
for i in y:
    score = (10000-i)**2
    resultat += score
print(resultat/len(y))
print(math.sqrt(resultat/len(y)))

# Scaler

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

# Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Lasso, LinearRegression


forest = RandomForestRegressor(n_estimators=100, max_leaf_nodes=1000, max_depth=50)
KNN = KNeighborsRegressor(n_neighbors=10)
lasso = Lasso(max_iter=5000,positive=True, fit_intercept=False, )
GBR = GradientBoostingRegressor(
    n_estimators=100, 
    learning_rate=0.1,
    max_depth=3)
    
ensemble = StackingRegressor(
    estimators = [('GBR', GBR), ("lasso", lasso), ('forest', forest)],
    final_estimator = lasso,
                n_jobs=-1)

In [ ]:
from tensorflow.keras import Sequential, layers


In [ ]:
# Add Estimator
pipe = make_pipeline(ensemble)

# Train pipeline
pipe.fit(X_train,y_train)

# Make predictions
#pipe.predict(X_test.iloc[0:2])

# Cross validate pipeline
#cross_val_score(ensemble, X_test, y_test, cv=5, scoring='r2').mean()
cross_val_score(ensemble, X_test, y_test, cv=5, scoring='neg_mean_squared_error').mean()

In [ ]:
math.sqrt(30990655)